In [ ]:
#Import dependencies 
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import json
import requests
from scipy.stats import linregress
import scipy.stats as stats

In [ ]:
#Read the csv files
olympics_df = pd.read_csv('Resources/athlete_events.csv')
host_cities_df = pd.read_csv('Resources/olym.csv')
arrivals_df = pd.read_csv('Resources/INT.ARVL.csv')
departures_df = pd.read_csv('Resources/INT.DPRT.csv')
expenditures_df = pd.read_csv('Resources/INT.XPND.csv')

In [ ]:
#Jumaan start here (Question 1)

How does hosting the olympics affect tourism for the host country in terms of international visitors and total expenditures?

In [ ]:
#Get the rows for the last 6 cities that hosted olympics
filtered_data = host_cities_df.iloc[45:]

#Get rid of unnecessary columns
last_six_hosts = filtered_data[['City', 'Country', 'Year']]

In [ ]:
#Rename country in arrivals dataset if different from host cities dataset
arrivals_df = arrivals_df.replace('Russian Federation', 'Russia')

#Rename column to allow merging
arrivals_df = arrivals_df.rename(columns={'Country Name': 'Country'})

In [ ]:
#Merge Datasets
combined_df = last_six_hosts.merge(arrivals_df, on='Country', how='left')

#Get rid of unnecessary columns
combined_df = combined_df.drop(['Country Code', 'Indicator Name', 'Indicator Code'], axis=1)

#Drop NaN columns
combined_df = combined_df.dropna(axis=1)

In [ ]:
#Create a function that calculates percent change
def percent_change(value_1, value_2):
    return (value_2 - value_1)/value_1 * 100

#Create a new data frame that holds the host countries and the arrival data for the year prior to hosting and the year they hosted
arrival_change_df = pd.DataFrame({
    'Country': combined_df['Country'],
    'Host Year': combined_df['Year'],
    'Arrivals before Host Year': '',
    'Arrivals on Host Year': '',
    'Percent Change': ''
    
})

arrival_data_prior_host_year = [combined_df.iloc[0, 13], combined_df.iloc[1, 15], combined_df.iloc[2, 17], \
                            combined_df.iloc[3, 19], combined_df.iloc[4, 21], combined_df.iloc[5, 23]]

arrival_data_on_host_year = [combined_df.iloc[0, 14], combined_df.iloc[1, 16], combined_df.iloc[2, 18], \
                            combined_df.iloc[3, 20], combined_df.iloc[4, 22], combined_df.iloc[5, 24]]

arrival_change_df['Arrivals before Host Year'] = arrival_data_prior_host_year

arrival_change_df['Arrivals on Host Year'] = arrival_data_on_host_year

arrival_change_df['Percent Change'] = percent_change(arrival_change_df['Arrivals before Host Year'], arrival_change_df['Arrivals on Host Year'])

arrival_change_df

In [ ]:
#Line graph for Arrivals

#x-axis
list_of_years = [int(i) for i in combined_df.iloc[:,13:].columns.tolist()]

#y-axis
italy_arrivals = combined_df.iloc[0,13:]
china_arrivals = combined_df.iloc[1,13:]
canada_arrivals = combined_df.iloc[2,13:]
uk_arrivals = combined_df.iloc[3, 13:]
russia_arrivals = combined_df.iloc[4, 13:]
brazil_arrivals = combined_df.iloc[5, 13:]

#set figure size
plt.figure(figsize=(15, 10))

#plots
plt.plot(list_of_years,italy_arrivals,label='Italy')
plt.plot(list_of_years,china_arrivals,label='China')
plt.plot(list_of_years,canada_arrivals,label='Canada')
plt.plot(list_of_years,uk_arrivals,label='UK')
plt.plot(list_of_years,russia_arrivals,label='Russia')
plt.plot(list_of_years,brazil_arrivals,label='Brazil')

#highlight the hosted year point
plt.plot(list_of_years[1], italy_arrivals[1], 'ko')
plt.plot(list_of_years[3],china_arrivals[3], 'ko')
plt.plot(list_of_years[5],canada_arrivals[5], 'ko')
plt.plot(list_of_years[7],uk_arrivals[7], 'ko')
plt.plot(list_of_years[9],russia_arrivals[9], 'ko')
plt.plot(list_of_years[11],brazil_arrivals[11], 'ko')

#formatting
plt.title('Years vs. International Arrivals')
plt.xlabel('Years')
plt.ylabel('Number of Arrivals')
plt.xticks(list_of_years, rotation=90)
plt.legend()

plt.show()

In [ ]:
#Rename country in expenditures dataset if different from host cities dataset
expenditures_df = expenditures_df.replace('Russian Federation', 'Russia')

#Rename column to allow merging
expenditures_df = expenditures_df.rename(columns={'Country Name': 'Country'})

In [ ]:
#Merge Datasets
combined_df = last_six_hosts.merge(expenditures_df, on='Country', how='left')

#Get rid of unnecessary columns
combined_df = combined_df.drop(['Country Code', 'Indicator Name', 'Indicator Code'], axis=1)

#Drop NaN columns
combined_df = combined_df.dropna(axis=1)

In [ ]:
#Create a new data frame that holds the host countries and the expenditures data for the year prior to hosting, the year they hosted and the year after hosting 
expenditure_change_df = pd.DataFrame({
    'Country': combined_df['Country'],
    'Host Year': combined_df['Year'],
    'Expenditures before Host Year': '',
    'Expenditures on Host Year': '',
    'Percent Change': ''
    
})

expenditure_data_prior_host_year = [combined_df.iloc[0, 13], combined_df.iloc[1, 15], combined_df.iloc[2, 17], \
                            combined_df.iloc[3, 19], combined_df.iloc[4, 21], combined_df.iloc[5, 23]]

expenditure_data_on_host_year = [combined_df.iloc[0, 14], combined_df.iloc[1, 16], combined_df.iloc[2, 18], \
                            combined_df.iloc[3, 20], combined_df.iloc[4, 22], combined_df.iloc[5, 24]]

expenditure_change_df['Expenditures before Host Year'] = expenditure_data_prior_host_year

expenditure_change_df['Expenditures on Host Year'] = expenditure_data_on_host_year

expenditure_change_df['Percent Change'] = percent_change(expenditure_change_df['Expenditures before Host Year'], expenditure_change_df['Expenditures on Host Year'])

expenditure_change_df

In [ ]:
# Extract individual groups
before_host = expenditure_change_df['Expenditures before Host Year']
on_host = expenditure_change_df['Expenditures on Host Year']

# Numbers of pairs of bars
N = 6

# Position of bars on x-axis
position = np.arange(N)

# Figure size
plt.figure(figsize=(15,10))

# Width of a bar 
width = 0.2

# Plotting
plt.bar(position, before_host, width, label='Before Hosting')
plt.bar(position + width, on_host, width, label='Year of Hosting')

plt.xlabel('Host Country')
plt.ylabel('US Dollars ($)')
plt.title('Olympics Host Country vs. International Expenditures')

# xticks()
plt.xticks(position + width/ 2, expenditure_change_df['Country'])

# Finding the best position for legends and putting it
plt.legend(loc='best')
plt.show()

# Question 1 Report

The Olympics is a time of competition and national pride. People around the world arrive at the Olympics to support their nation's athletes throughout many sporting events. While the olympics are catered around its athletic events and fans, it is also very much about business. Host cities invest into the Olympics heavily in hopes of making sizable gains for both the local and national economy through tourism, marketing, etc. In this analysis, we look at the influence of the Olympics on the host country's international arrival data as well as international expenditure data.

Looking at the timeline graph outlining the international arrival data for countries that hosted since 2006, we can see that majority of the countries experienced an uptick in their number of international arrivals on the year they hosted aside from China. China's number of international arrivals dropped by 3% compared to the year before. However, there is much needed context to explain this change. Leading up to the 2008 Summer Olympics, there were numerous concerns regarding human rights violation committed by China during the civil unrest in Tibet that led high-profile politicians and celebrities to boycott the games. Additionally, the level of air pollution in Beijing came under scrutiny due to the hazard it posed to the competing athletes and the general public. The financial crisis of 2007-2008 was also a factor since it was a severe worldwide economic crisis limiting an individual's spending ability.

The international expenditure data, on the other hand, tells us a different story of the macroeconomy during the time of Olympics and the year after. Nations such as Italy, China, Canada, and the United Kingdom experienced an increase in their international expenditures the year they hosted the Olympics as opposed to Russia and Brazil. Russia hosted the Olympics in 2014, a time at which they were experiencing their own financial crisis. Due to the fall in price of oil, a major export of Russia, a sharp devaluation of the Russian ruble took place. Similarily, Brazil also faced an economic crisis in 2015, a year before hosting the Olympics. This occured as a result of a dispute in the legitmacy of the Brazilian elections leading to dissatification with the government by the general public. There were also concerns about the Zika virus that raised fears regarding its potential effect on competing athletes and visitors. 

In [ ]:
#Jumaan end here

In [ ]:
#Timmy start here (Question 2)

Does hosting olympics affects the travel habit of the people of that country? ( People see lots of different cultures coming to them and want to explore those cultures by themselves) (Timmy)

In [ ]:
olympics_df = olympics_df.replace(['Antwerpen', 'Torino', 'Athina', 'Roma', 'Moskva', 'Sankt Moritz'], ['Antwerp', 'Turin', 'Athens', 'Rome', 'Moscow', 'St. Moritz'])

In [ ]:

host_cities_df = host_cities_df[["City", "Country"]]
host_cities_df.head()

In [ ]:
olympics_country_df = pd.merge(olympics_df, host_cities_df, how="left", on=["City"])
olympics_country_df=olympics_country_df.drop_duplicates()
# remove duplicates
olympics_country_df.head()

In [ ]:
#this allows me to focus on each city that hosted the games, for future merging
cities_only = olympics_country_df.drop_duplicates(subset = ['City'])
cities_only.head()

In [ ]:
#check if it's the correct number
len(cities_only)

In [ ]:
#the only columns important to me
edited_cities_df = cities_only[["City", "Country","Year", "Season"]]
edited_cities_df = edited_cities_df.reset_index(drop=True)


In [ ]:
games_with_data = edited_cities_df.loc[edited_cities_df['Year'] > 1960]


In [ ]:
departures_df.columns

In [ ]:
#basically there is no data til 1995 for departures which is a bummer, so we are dropping coloumns
departures_df.drop(departures_df.iloc[:, 4:39], inplace = True, axis = 1)
departures_df.head()

In [ ]:
#rename coloumn for future merging
departures_df
departures_df = departures_df.rename(columns={'Country Name': 'Country'})

In [ ]:
#change country names in data so it matches
games_with_data = games_with_data.replace(['Russia', 'West Germany', 'South Korea', 'Soviet Union'],
                                          ['Russian Federation', 'Germany', 'Korea, Rep.', 'Russian Federation'])


In [ ]:
#reseting the index for finding specific rows
games_with_data = games_with_data.reset_index(drop=True)

In [ ]:
#no data on Yugoslavia so bye
games_with_data = games_with_data.drop([16])


In [ ]:
#merging data
olympics_host_depart_df = pd.merge(games_with_data, departures_df, how="left", on=["Country"])
olympics_host_depart_df = olympics_host_depart_df.drop_duplicates()

In [ ]:
#departure information basically starts at 1995
olympics_hostingclose_df=olympics_host_depart_df.loc[olympics_host_depart_df['Year'] > 1995]


In [ ]:
#the cities/countries that hosted the olympics and has depart info on them
olympics_hostingclose_df = olympics_hostingclose_df.reset_index(drop=True)
olympics_final_df = olympics_hostingclose_df.drop([9])
olympics_final_df

In [ ]:
olympics_final_df.set_index('Country')


In [ ]:
# starting to try to work data in a way thats good for graphs
for_graph = olympics_final_df

In [ ]:
#simplify drop a bunch of coloums
for_graph = for_graph.drop(['City', 'Year','Season', 'Country Code', 'Indicator Name',
       'Indicator Code', '2019', '2020'], axis=1)

In [ ]:
for_graph.set_index('Country')

In [ ]:
for_graph = for_graph.astype(dict.fromkeys(for_graph.columns[1:25], float))

In [ ]:
#figureout how to do a for loop for this
# do pct chabfe for each and also fill in the first cell since it will be NA

#summer game
uk = for_graph.iloc[0]
uk_years = uk[1 : 25]
uk_pct_hoststart = uk[18: 22].pct_change() * 100 
uk_pct_hoststart = uk_pct_hoststart.fillna(0)

#winter game
usa_1 = for_graph.iloc[1]
usa_1_years = usa_1[1 : 25]
usa_1_pct_hoststart = usa_1[8: 12].pct_change() * 100
usa_1_pct_hoststart = usa_1_pct_hoststart.fillna(0)

#summer game
aus = for_graph.iloc[2]
aus_years = aus[1 : 25]
aus_pct_hoststart = aus[6 : 10].pct_change() * 100
aus_pct_hoststart = aus_pct_hoststart.fillna(0) 

#summer game
usa_2 = for_graph.iloc[3]
usa_2_years = usa_2[1 : 25]
usa_2_pct_hoststart = usa_2[2: 6].pct_change() * 100
usa_2_pct_hoststart = usa_2_pct_hoststart.fillna(0)

#winter game
rus = for_graph.iloc[4]
rus_years = rus[1 : 25]
rus_pct_hoststart = rus[20 : 24].pct_change() * 100
rus_pct_hoststart = rus_pct_hoststart.fillna(0)

#winter game
jap = for_graph.iloc[5]
jap_years = jap[1 : 25]
jap_pct_hoststart = jap[4 : 8].pct_change() * 100
jap_pct_hoststart = jap_pct_hoststart.fillna(0)

#winter game
ita = for_graph.iloc[6]
ita_years = ita[1 : 25]
ita_pct_hoststart = ita[12 : 16].pct_change() * 100
ita_pct_hoststart = ita_pct_hoststart.fillna(0)

#summer game
chi = for_graph.iloc[7]
chi_years = chi[1 : 25]
chi_pct_hoststart = chi[14 : 18].pct_change() * 100
chi_pct_hoststart = chi_pct_hoststart.fillna(0)

#summer game
bra = for_graph.iloc[8]
bra_years = bra[1 : 25]
bra_pct_hoststart = bra[22 : 26].pct_change() * 100
bra_pct_hoststart = bra_pct_hoststart.fillna(0)

#winter game
can = for_graph.iloc[9]
can_years = can[1 : 25]
can_pct_hoststart = can[16 : 20].pct_change() * 100
can_pct_hoststart = can_pct_hoststart.fillna(0)


In [ ]:
#pecercent change graph of when each country hosted the oylmpics
x_axis = range(len(uk_pct_hoststart)) 


plt.plot(x_axis, ita_pct_hoststart)
plt.plot(x_axis, can_pct_hoststart)
plt.plot(x_axis, usa_1_pct_hoststart)
plt.plot(x_axis, jap_pct_hoststart)
plt.plot(x_axis, rus_pct_hoststart)

plt.legend(['Italy 2006','Canada 2010','USA 2002','Japan 1998', 'Russia 2014'],
           loc='center left', bbox_to_anchor=(1, 0.82))

plt.title("Winter Olympic Games")
plt.ylabel("Percent Change %")
plt.xlabel("Years Since Hosting")
plt.savefig("graphs_Timmy/pct_change_winter")
plt.show()

In [ ]:
#fillin in the last value because it isnt worthing throuwing out that row
bra_pct_hoststart["2019"] = np.nan
bra_pct_hoststart = bra_pct_hoststart.fillna(method="ffill")

In [ ]:
#pecercent change graph of when each country hosted the oylmpics
plt.plot(x_axis, uk_pct_hoststart)
plt.plot(x_axis, aus_pct_hoststart)
plt.plot(x_axis, usa_2_pct_hoststart)
plt.plot(x_axis, chi_pct_hoststart)
plt.plot(x_axis, bra_pct_hoststart)

plt.legend(['United Kingdom 2012','Australia 2000','USA 1996','China 2008', 'Brazil 2016'],
           loc='center left', bbox_to_anchor=(1, 0.82))

plt.title("Summer Olympic Games")
plt.ylabel("Percent Change %")
plt.xlabel("Years Since Hosting")
plt.savefig("graphs_Timmy/pct_change_summer")
plt.show()

In [ ]:
years = [int(i) for i in for_graph.iloc[:,1:].columns.tolist()]
years_early = [int(i) for i in for_graph.iloc[:,1:].columns.tolist()]

#years.drop(years.index[[1,2,3,4,5,6,7,8,9,10,11]], inplace=True)
#del years[0:13]
del years_early[13:25]


In [ ]:
china = for_graph.iloc[7]
china_postyears = china[14 : 25]
china_preyears = china[1 : 14]
china_pct_hoststart = china[13 : 25].pct_change() * 100


In [ ]:
china_preyears = china_preyears.astype(float)
china_postyears = china_postyears.astype(float)



In [ ]:
#neccsary
years = years[0:24]
    
years_ints = []
for item in years:
    years_ints.append(int(item))

china = china[1:25]

china_ints = []
for x in china:
    china_ints.append(int(x))

In [ ]:
#when in doubt put it in a new dataframe, helpgul tip the rows must match
china_df = pd.DataFrame({"Total Years":years_ints, 
                        "Total Departs":china_ints})



In [ ]:


early_years = china_df['Total Years'][0:13]
early_departs = china_df['Total Departs'][0:13]

post_years = china_df['Total Years'][14:24]
post_departs = china_df['Total Departs'][14:24]


In [ ]:
x_values = china_df['Total Years']
y_values = china_df['Total Departs']


plt.scatter(x_values, y_values, marker = "o", color = "blue",edgecolor = "black")

#overall
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

#early years1
(slope1, intercept1, rvalue1, pvalue1, stderr1) = linregress(early_years, early_departs)
regress_values1 = early_years * slope1 + intercept1
line_eq1 = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))

#later years 2
(slope2, intercept2, rvalue2, pvalue2, stderr2) = linregress(post_years, post_departs)
regress_values2 = post_years * slope2 + intercept2
line_eq2 = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))


#plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"g-")
plt.plot(early_years,regress_values1,"r-")
plt.plot(post_years,regress_values2,"b-")

plt.grid()
plt.xlabel("Years")
plt.ylabel("Departures from China(mil)")
plt.title("Trend Line")
plt.savefig("graphs_Timmy/China_rate_change")
plt.show()

In [ ]:
round(slope2)/round(slope1)

   The Olympics is a massive international event that brings together around two hundred countries for an exciting 
two weeks of athletic events. However, our thesis states that hosting the Olympics has much bigger effects that 
go past just the two week period. My section is focused on how it impacts the citizens of the host country, 
specifically in the focus of tourism. We chose to focus on tourism because one of the underlying messages of 
the Olympics is of unity, of exploration, and to push oneself. This emphasis is more hypothetically more heavily felt 
by the host countries citizens that are seeing the best of their own country on this international stage. The impact 
should be a noticeable increase in citizens going on international trips to visit other countries, since they were 
inspired to explore the world. In the following paragraph we shall see if this holds up, and how I worked through the
data to find said results.
   I’ve learned that cleaning the data, and completely understanding is not only fundamental to analyzing data, but also fascinating. The routes each of us took and branched from to find answers for each of our own questions shows the flexibility and the power of this first crucial step to analyzing any data. I digress, the main highlights, are whittling down the massive olympics data to only have the city, the country, the year, and season of the game for the past hundred years, which is forty-two places. Then by merging left and constricting for the years on when we have information on the number of departures, which results to ten cities from 1995 to 2018. Perfectly, that leaves us with five Winter Olympic Games and five Summer Olympic Games to analyze. To graph some of the data I discovered. I focused one section at looking at all the summer and winter games on their own graphs, and I believe the best way to visualize the changes over time would be a multi line graph to compare the trajectories in a succinct way. The best way is to have the year of each country hosting as the start date and then make the each timeline relative to it’s own growth, so I decided to make this graph focus on the percent change for the next three years to see if there was an increase or decrease in departures. What I found seems to be slightly positive to an increase in departures post hosting the Olympics, however it’s more inconclusive, since the results are generally all over the place without any similar patterns matching up. Which is frustrating, but fair since, the world is a complex place and not everything is going to be so simple for a cause and effect.
   The second graph, I decided to focus on one country and see if a linear regression could be insightful to see if the rate of departures changed from pre hosting the Olympics to post. I chose China as the candidate because in just a few months it will be hosting the Olympics again, so it would be fun to see if any the linear regression would still hold true. (Though with the pandemic about, probably not.) I created the linear regressions and found the slope after hosting was three point nine times the rate of departures before hosting the Olympics. This could be a strong result for use to be able to slightly reject the null hypothesis.
   This increase of international tourism and travel can be from the impact the Olympic Games had on individual people as well as corporations to pursue connections outside of China. As China is one of the booming economies the more business fields the countries companies can partner with the more travel to said countries would most likely follow. That is just one of several reasons why the departure rate increased, still I’m happy that we can reject the null hypothesis in some capacity!

In [ ]:
#Timmy end here

In [ ]:
#Raj start here (Question 3)

In [ ]:
#Raj end here

In [ ]:
#Fatma start here (Question 4)

IS THERE A STATISTICALLY SIGNIFICANT DIFFERENCE BETWEEN MEDALS WON PRIOR TO HOSTING OLYMPICS AND AFTER HOSTING?

In [ ]:
# quick look on olympics data
olympics_df.head()

In [ ]:
# look at host cities data
host_cities_df

# Clean up misspelled words in the dataframe

DATA CLEAN-UP

In [ ]:
# get only the city and country column from the host city data
host_cities_df = host_cities_df[["City", "Country"]]
host_cities_df.head()

# find the unique city and country names in the host city data frame, and use them to correct spelling mistakes
z=host_cities_df["City"].unique()
y=host_cities_df["Country"].unique()
z
y

In [ ]:
#replace the west germany and nazi germany with germany and soviet union with russia in the host_cities file

host_cities_df = host_cities_df.replace({'Germnay' : 'Germany', 'West Germany': 'Germany', 'Nazi Germany': 'Germany' , 'Soviet Union' : 'Russia'})

#replace the misspelled city names with the correct ones based on the host cities file

olympics_c_df= olympics_df.replace({'Athina':'Athens', 'Antwerpen':'Antwerp', 'Sankt Moritz':'St. Moritz', 'Roma':'Rome', 'Moskva':'Moscow', 'Torino':'Turin'})

#merge olympics and host cities data

olympics_country_df = pd.merge(olympics_c_df, host_cities_df, how="left", on=["City"])

# remove duplicates from olympics data
olympics_country_df = olympics_country_df.drop_duplicates()

# look at data quickly
olympics_country_df.head()

In [ ]:
# leave only the columns that are relevant to the analysis
olympics_country_df = olympics_country_df[["Name", "Team", "Year", "Season", "Sport", "Event", "Medal", "Country"]]
olympics_country_df.shape

In [ ]:
# check if any NaN is left in the Country column. Shape should be equal to olympics_country_df.if there is no NaN left nothing 
# will drop and the row numbers should stay same.
x = olympics_country_df.dropna(subset=["Country"])
x.shape

# The medal column has NaN for no medal won. Replace all with 0

In [ ]:
# replace all the NaN in the Medal column with 0
values = {"Medal" : 0}
olympics_country_df = olympics_country_df.fillna(value=values)
olympics_country_df = pd.DataFrame(olympics_country_df)
olympics_country_df.head()

# Make a dataframe that shows only teams for the years they hosted and remove any rows when no medal was won

CREATE A NEW DATAFRAME THAT INCLUDES INFORMATION ONLY ABOUT THE HOSTING COUNTRY THE YEAR THEY HOSTED

In [ ]:
# Make a new df where the team is from the host country

host_team = olympics_country_df.loc[olympics_country_df["Team"] == olympics_country_df["Country"]]
host_team.head()

In [ ]:
# remove the columns with no medal win

medal_win = host_team.loc[host_team["Medal"] != 0]

# drop any duplicate row if there is any
medal_win = medal_win.dropna()

# keep only columns of interest
medal_win = medal_win[[ "Team", "Year", "Medal"]]

In [ ]:
# how many medals host countries won, do groupby

medal_win_host = medal_win.groupby(["Team", "Year"]).count()
medal_win_host

# reset index 

host_year_medal_count = medal_win_host.reset_index()
host_year_medal_count.head()

In [ ]:
# from my corrected host cities dataframe find the unique countries
host_countries = host_cities_df["Country"].unique()
host_countries


# Make a dataframe that shows all the teams that hosted olympics anytime, remove rows when no medal was won, count the total medal won for that country in that year

CREATE A NEW DATAFRAME WITH TEAMS THAT HOSTED THE OLYMPICS AT ANY TIMES

In [ ]:
# clean the olympics dataframe only that it has teams that were also hosts at anytime during the olympics

only_teams_that_hosted = olympics_country_df[olympics_country_df["Team"].isin(olympics_country_df["Country"])]
only_teams_that_hosted.head()

In [ ]:
d =only_teams_that_hosted.groupby(["Team", "Year"]).count()
finland = d.loc["Finland"]
finland.plot.bar(y="Medal", title="Olympics Participation from Finland")
plt.show()

In [ ]:
#remove the columns with no medal win

medal_win_by_team = only_teams_that_hosted.loc[only_teams_that_hosted["Medal"] != 0]
medal_win_by_team = medal_win_by_team.dropna()

#this is different from the medal win dataframe in that this one includes all the years the hosting country participated
#keep only relevant columns

medal_win_by_team = medal_win_by_team[["Team", "Year", "Medal"]]

In [ ]:
# group by team and count the medal wins of the team during the entire time (not only when it is host)

year_medal_count = medal_win_by_team.groupby(["Team", "Year"]).count()
year_medal_count.head()


# Make bar graphs showing how many medals each contry won. Color the year of hosting red

MAKE BAR GRAPHS SHOWING MEDAL COUNTS FOR HOST COUNTRIES THROUGHOUT THE YEARS

In [ ]:
# for loop for each team 

team = only_teams_that_hosted["Team"].unique()
team = team.tolist()
team

for team in team:
    
    teams = year_medal_count.loc[team]
    
    bar = host_year_medal_count.loc[host_year_medal_count["Team"] == team ]["Year"].tolist()
    
    colors = ["red" if x in bar else "yellow" for x in teams.index]
    
    teams.plot.bar(y="Medal", title=f"Olympic medal counts for {team} ", color=colors)
  
    plt.show()
    
    




DO THE GRAPHS AND HYPOTHESIS TESTING FOR SUMMER OLYMPIC ONLY 

In [ ]:
# get the medal wins only for summer olympics for host teams


summer_wins = host_team.loc[host_team["Medal"] != 0]
summer_wins = summer_wins.dropna()
summer_wins

summer_wins = summer_wins.loc[summer_wins["Season"] == "Summer"]
summer_wins = summer_wins[[ "Team", "Year", "Medal"]]
summer_wins

In [ ]:
# medal counts for teams during hosting
summer_win_host = summer_wins.groupby(["Team", "Year"]).count()
summer_win_host

host_year_medal_count_summer = summer_win_host.reset_index()
host_year_medal_count_summer.head()

In [ ]:
# divide the only teams that hosted dataframe into summer

only_teams_that_hosted_summer = only_teams_that_hosted.loc[only_teams_that_hosted["Season"] == "Summer"]
only_teams_that_hosted_summer.head()

In [ ]:
#remove the columns with no medal win

medal_win_by_team_summer = only_teams_that_hosted_summer.loc[only_teams_that_hosted_summer["Medal"] != 0]
medal_win_by_team_summer= medal_win_by_team_summer.dropna()
medal_win_by_team_summer = medal_win_by_team_summer[["Team", "Year", "Medal"]]

In [ ]:
# group by team and count the medal wins of the team during the entire time (not only when it is host)

year_medal_count_summer = medal_win_by_team_summer.groupby(["Team", "Year"]).count()
year_medal_count_summer.head()

In [ ]:
# for loop for each team 

team = only_teams_that_hosted_summer["Team"].unique()
team = team.tolist()
team

for team in team:
    
    teams = year_medal_count_summer.loc[team]
    
    bar = host_year_medal_count_summer.loc[host_year_medal_count_summer["Team"] == team ]["Year"].tolist()
    
    colors = ["red" if x in bar else "yellow" for x in teams.index]
    
    teams.plot.bar(y="Medal", title=f"Olympic medal counts for {team} ", color=colors)
    
    plt.show()
    


# Calculate Ttest for checking the null hypothesis that states "The mean medal win for a country before and after hosting the Olympics is NOT different"

HYPOTHESIS TESTING

Null Hypothesis :For countries that hosted the olympics the average medal counts are the same before and after being the host 

In [ ]:
# create a list of dataframes that will show the country, year they played and medals they won
# first remove all countries that hosted for more than ones. 

j =year_medal_count_summer.reset_index()
t= host_year_medal_count_summer


t["Team"].value_counts() == 1
r = t.drop_duplicates(subset = "Team", keep=False)
r

# make a list that contains these countries
team_summer = r["Team"].tolist()

In [ ]:
# now make a for loop that will show the country and medal wins the years after they hosted.
team_list_after = []

for team in team_summer:
    
    df = j.loc[(j["Team"] == team) & (j["Year"] > r[r['Team']==team]['Year'].values[0])]
    team_list_after.append(df)
    
    
    
team_list_after[0]

In [ ]:
# now make a for loop that will show the country and medal wins the years before they hosted.


team_list_before = []

for team in team_summer:
    
    df = j.loc[(j["Team"] == team) & (j["Year"] < r[r['Team']==team]['Year'].values[0])]
    team_list_before.append(df)
    
    
    
team_list_before[0]["Medal"]

In [ ]:
# calculate t test for each country data obtained from the for loops above
#make a for loop for each team

teams = []
for i in range(len(team_list_after)):

    test = stats.ttest_ind(team_list_after[i].Medal, team_list_before[i].Medal, equal_var=False)
    teamss = team_list_after[i]["Team"].unique().tolist()
    teams.append(teamss)
    
    print(f"Ttest result for {teams[i]}: {test}")

# We were able to reject our null hypothesis for Belgium, Canada, China, Finland, Germany,Japan, Netherlands, South Korea and Spain. 
# We failed to reject the null hypothesis for Italy and Mexico.
# We did not have enough data to perform the test on Brazil.

CHECK WHETHER THE DECREASE IN MEDAL COUNTS IN FINLAND IS DUE TO DECREASED PARTICIPATION 

Null hypothesis : The average of athletes that participated in the olympics from Finland is the same before and after Finland hosted the olympics

In [ ]:
d =only_teams_that_hosted_summer.groupby(["Team", "Year"]).count()
finland = d.loc["Finland"]
finland.plot.bar(y="Medal", title="Olympics Participation from Finland")
plt.show()
finland


In [ ]:
participation_before = finland.loc[(finland.index < 1952)]

participation_after = finland.loc[(finland.index > 1952)]

ttest = stats.ttest_ind(participation_before.Medal, participation_after.Medal, equal_var=False)
ttest

print("We failed to reject the null hypothesis. Therefor we cannot say the average participation was different before and after 1952")

We were able to reject our null hypothesis for Belgium, Canada, China, Finland, Germany,Japan, Netherlands, South Korea and Spain. 
We failed to reject the null hypothesis for Italy and Mexico.
We did not have enough data to perform the test on Brazil.
The decrease of Finland's Medal counts does not seem to be due to a decrease in participation.

In [ ]:
#Fatma end here